# 详解Python的装饰器
http://python.jobbole.com/86717

## 为什么需要装饰器

我们假设你的程序实现了say_hello()和say_goodbye()两个函数。



In [4]:
def say_hello():
    print("hello")
    
def say_goodbye():
    print("goodbye!")
    
if __name__ == '__main__':
    say_hello()
    say_goodbye()    

hello
goodbye!


In [6]:
def debug():
    import inspect
    caller_name = inspect.stack()[1][3]
    print("[DEBUG]: enter {}()".format(caller_name))   
 
def say_hello():
    debug()
    print("hello!")
 
def say_goodbye():
    debug()
    print("goodbye!")
 
if __name__ == '__main__':
    say_hello()
    say_goodbye()

[DEBUG]: enter say_hello()
hello!
[DEBUG]: enter say_goodbye()
goodbye!


### 1.0 早期版本 （不传参)

In [19]:
def debug(func):
    def wrapper():
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func()
    return wrapper
 
def say_hello():
    print("hello!")
 
say_hello = debug(say_hello)# 添加功能并保持原函数名不变
say_hello()

[DEBUG]: enter say_hello()
hello!


### 2.0 早期版本（传参）

In [26]:
def debug(func):
    def wrapper(something):  # 指定一毛一样的参数
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func(something)
    return wrapper  # 返回包装过函数
 
@ debug
def say(something):
    print("hello {}!".format(something))

say('say something')

[DEBUG]: enter say()
hello say something!


### 2.5 早期版本（传任意参）
Python提供了可变参数*args和关键字参数**kwargs，有了这两个参数，装饰器就可以用于任意目标函数了。

In [32]:
def debug(func):
    def wrapper(*args, **kwargs):  # 指定宇宙无敌参数
        print("[DEBUG]: enter {}()".format(func.__name__))
        print('Prepare and say...',)
        return func(*args, **kwargs)
    return wrapper  # 返回
 
@ debug
def say(something):
    print("hello {}!".format(something))
    
say('something')

[DEBUG]: enter say()
Prepare and say...
hello something!


### 3.0 高级版本 (传参)

假设我们前文的装饰器需要完成的功能不仅仅是能在进入某个函数后打出log信息，而且还需指定log的级别，那么装饰器就会是这样的。

In [37]:
def logging(level):
    def wrapper(func):
        def inner_wrapper(*args, **kwargs):
            print("[{level}]: enter function {func}()".format(
                level=level,
                func=func.__name__))
            return func(*args, **kwargs)
        return inner_wrapper
    return wrapper
 
@logging(level='INFO')
def say(something):
    print("say {}!".format(something))
 
# 如果没有使用@语法，等同于
# say = logging(level='INFO')(say)
 
@logging(level='DEBUG')
def do(something):
    print('do {}...'.format(something))
 
if __name__ == '__main__':
    say('hello')
    do("my work")

[INFO]: enter function say()
say hello!
[DEBUG]: enter function do()
do my work...


你可以这么理解，当带参数的装饰器被打在某个函数上时，比如@logging(level='DEBUG')，它其实是一个函数，会马上被执行，只要这个它返回的结果是一个装饰器时，那就没问题。

### 基于类实现的装饰器

装饰器函数其实是这样一个接口约束，它必须接受一个callable对象作为参数，然后返回一个callable对象。

在Python中一般callable对象都是函数，但也有例外。只要某个对象重载了__call__()方法，那么这个对象就是callable的。

In [39]:
class Test():
    def __call__(self):
        print('call me!')
 
t = Test()
t()  # call me

call me!


像__call__这样前后都带下划线的方法在Python中被称为内置方法，有时候也被称为魔法方法。重载这些魔法方法一般会改变对象的内部行为。

上面这个例子就让一个类对象拥有了被调用的行为

回到装饰器上的概念上来，装饰器要求接受一个callable对象，并返回一个callable对象（不太严谨，详见后文）。那么用类来实现也是也可以的。我们可以让类的构造函数__init__()接受一个函数，然后重载__call__()并返回一个函数，也可以达到装饰器函数的效果。

In [44]:
class logging(object):
    def __init__(self, func):
        self.func = func
 
    def __call__(self, *args, **kwargs):
        print("[DEBUG]: enter function {func}()".format(
            func=self.func.__name__))
        return self.func(*args, **kwargs)
    
@logging
def say(something):
    print("say {}!".format(something))
    
if __name__=='__main__':
    say('Hello Alvin')

[DEBUG]: enter function say()
say Hello Alvin!


### 带参数的类装饰器

如果需要通过类形式实现带参数的装饰器，那么会比前面的例子稍微复杂一点。那么在构造函数里接受的就不是一个函数，而是传入的参数。通过类把这些参数保存起来。然后在重载__call__方法是就需要接受一个函数并返回一个函数。

In [47]:
class logging(object):
    def __init__(self, level='INFO'):
        self.level = level
        
    def __call__(self, func): # 接受函数
        def wrapper(*args, **kwargs):
            print("[{level}]: enter function {func}()".format(
                level=self.level,
                func=func.__name__))
            func(*args, **kwargs)
        return wrapper  #返回函数
 
@logging(level='INFO')
def say(something):
    print("say {}!".format(something))
    
if __name__=='__main__':
    say('Hello Alvin')

[INFO]: enter function say()
say Hello Alvin!


### 内置的装饰器

内置的装饰器和普通的装饰器原理是一样的，只不过返回的不是函数，而是类对象，所以更难理解一些。

#### @property
在了解这个装饰器前，你需要知道在不使用装饰器怎么写一个属性。

In [49]:
def getx(self):
    return self._x
 
def setx(self, value):
    self._x = value
    
def delx(self):
    del self._x
 
# create a property
x = property(getx, setx, delx, "I am doc for x property")

In [50]:
property()

#### @staticmethod，@classmethod

有了@property装饰器的了解，这两个装饰器的原理是差不多的。

@staticmethod返回的是一个staticmethod类对象

@classmethod返回的是一个classmethod类对象

他们都是调用的是各自的__init__()构造函数

In [51]:
class classmethod(object):
    """
    classmethod(function) -> method
    """    
    def __init__(self, function): # for @classmethod decorator
        pass
    # ...
class staticmethod(object):
    """
    staticmethod(function) -> method
    """
    def __init__(self, function): # for @staticmethod decorator
        pass
    # ...

In [52]:
# 装饰器的@语法就等同调用了这两个类的构造函数。

class Foo(object):
 
    @staticmethod
    def bar():
        pass
    
    # 等同于 bar = staticmethod(bar)

## 装饰器里的那些坑



### 位置错误的代码  (Note: need to review)

In [79]:
def html_tags(tag_name):
    print ('begin outer function.')
    def wrapper_(func):
        print ("begin of inner wrapper function.")
        def wrapper(*args, **kwargs):
            content = func(*args, **kwargs)
            print ("<{tag}>{content}</{tag}>".format(tag=tag_name, content=content))
#             return func(*args, **kwargs)
        print ('end of inner wrapper function.')
        return wrapper
    print('end of outer function')
    return wrapper_
 
@html_tags('b')
def hello(name='Toby'):
    return 'Hello {}!'.format(name)
 
print(hello())
print(hello('Alvin'))
print(hello('Andrew'))

begin outer function.
end of outer function
begin of inner wrapper function.
end of inner wrapper function.
<b>Hello Toby!</b>
None
<b>Hello Alvin!</b>
None
<b>Hello Andrew!</b>
None


In [71]:
# template code
def logging(level):
    def wrapper(func):
        def inner_wrapper(*args, **kwargs):
            print("[{level}]: enter function {func}()".format(
                level=level,
                func=func.__name__))
            return func(*args, **kwargs)
        return inner_wrapper
    return wrapper

In [78]:
def html_tags(tag_name):
    print ('begin outer function.')
    def wrapper_(func):
        print ("begin of inner wrapper function.")
        def wrapper(*args, **kwargs):
            content = func(*args, **kwargs)
            print ("<{tag}>{content}</{tag}>".format(tag=tag_name, content=content))
            return func(*args, **kwargs)
        print ('end of inner wrapper function.')
        return wrapper
    print('end of outer function')
    return wrapper_
 
@html_tags('b')
def hello(name='Toby'):
    return 'Hello {}!'.format(name)

 
print(hello())
print(hello('Alvin'))
print(hello('Andrew'))

begin outer function.
end of outer function
begin of inner wrapper function.
end of inner wrapper function.
<b>Hello Toby!</b>
Hello Toby!
<b>Hello Alvin!</b>
Hello Alvin!
<b>Hello Andrew!</b>
Hello Andrew!


### 错误的函数签名和文档

装饰器装饰过的函数看上去名字没变，其实已经变了。

In [125]:
def logging(func):
    def wrapper(*args, **kwargs):
        """print log before a function."""
        print( "[DEBUG]: enter {}()".format(func.__name__))
        return func(*args, **kwargs)
    return wrapper
 
@logging
def say(something):
    """say something"""
    print ("say {}!".format(something))
 
print( say.__name__, ' - ', say.__doc__)  # wrapper

wrapper  -  print log before a function.


In [126]:
#只要你想想装饰器的语法糖@代替的东西就明白了。@等同于这样的写法。
say = logging(say)

In [127]:
from functools import wraps
 
def logging(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        """print log before a function."""
        print("[DEBUG]: enter {}()".format(func.__name__)) 
        return func(*args, **kwargs)
    return wrapper
 
@logging
def say(something):
    """say something"""
    print("say {}!".format(something))
 
print( say.__name__, ' - ', say.__doc__)

say  -  say something


In [128]:
import inspect
# print(inspect.getargspec(say))  # deprecated in python 3
print(inspect.signature(say)) 
print()
print(inspect.getsource(say)) 

(something)

@logging
def say(something):
    """say something"""
    print("say {}!".format(something))



## 小结

Python的装饰器和Java的注解（Annotation）并不是同一回事，和C#中的特性（Attribute）也不一样，完全是两个概念。

装饰器的理念是对原函数、对象的加强，相当于重新封装，所以一般装饰器函数都被命名为wrapper()，意义在于包装。函数只有在被调用时才会发挥其作用。比如@logging装饰器可以在函数执行时额外输出日志，@cache装饰过的函数可以缓存计算结果等等。

而注解和特性则是对目标函数或对象添加一些属性，相当于将其分类。这些属性可以通过反射拿到，在程序运行时对不同的特性函数或对象加以干预。比如带有Setup的函数就当成准备步骤执行，或者找到所有带有TestMethod的函数依次执行等等。